In [32]:
import os 
import pandas as pd 
os.chdir("/mnt/BioHome/jreyna/jreyna/projects/dchallenge/")
import glob

In [33]:
# loading and concat all the data
data = []
for fn in glob.glob('results/main/loop_analysis/T1D_34012112_Gaulton/*/*/*/master.tsv'):
    
    dice_cline = fn.split('/')[4]
    study = fn.split('/')[5]
    eqtl_cline = fn.split('/')[6]
    
    df = pd.read_table(fn, header=0)
    df['dice_cline'] = dice_cline
    df['eqtl_cline'] = eqtl_cline
    df['study'] = study
    data.append(df)
    
data = pd.concat(data)
data.rename(columns={'5kb_gname': 'fivekb_gname'}, inplace=True)

In [34]:
# extract the looping genes 
fivekb_lgenes = []
eqtl_lgenes = []
cls_lgenes = []

for i, sr in data.iterrows():
    
    # fivekb related 
    genes = sr['fivekb_gname'].split(',')
    bools = sr['fivekb_loops'].replace('[', '').replace(']', '').replace(' ', '').split(',')
    final_genes = []
    for g, b in zip(genes, bools):
        if b == '1':
            final_genes.append(g)
    final_genes = ','.join(final_genes)
    fivekb_lgenes.append(final_genes)
    
    # eqtl related 
    genes = sr['eqtl_gnames'].split(',')
    bools = sr['eqtl_loops'].replace('[', '').replace(']', '').replace(' ', '').split(',')
    final_genes = []
    for g, b in zip(genes, bools):
        if b == '1':
            final_genes.append(g)
    final_genes = ','.join(final_genes)
    eqtl_lgenes.append(final_genes)
    
    
    
    # eqtl related 
    genes = sr['cls_gnames'].split(',')
    bools = sr['closest_loops'].replace('[', '').replace(']', '').replace(' ', '').split(',')
    final_genes = []
    for g, b in zip(genes, bools):
        if b == '1':
            final_genes.append(g)
    final_genes = ','.join(final_genes)
    cls_lgenes.append(final_genes)
    

    

In [35]:
data['five5kb_loop_gnames'] = fivekb_lgenes
data['eqtl_loop_gnames'] = eqtl_lgenes
data['cls_loop_gnames'] = cls_lgenes

In [36]:
new_order = ['rs_id',
 'dice_cline',
 'eqtl_cline',
 'study',
 'chr',
 'pos',
 'coloc_gname',
 'coloc_loops',
 'eqtl_loop_gnames',
 'cls_loop_gnames',
 'five5kb_loop_gnames',              
 'variant_id',
 'pp_H0_Coloc_Summary',
 'pp_H1_Coloc_Summary',
 'pp_H2_Coloc_Summary',
 'pp_H3_Coloc_Summary',
 'pp_H4_Coloc_Summary',
 'dist',
 'pvalue',
 'FDR',
 'slope_snp',
 'ref',
 'alt',
 'AC',
 'AF',
 'AN',
 'slope_se_snp',
 'slope_gwas',
 'slope_se_gwas',
 'pval_nominal',
 'SampleSize',   
 'eqtl_gnames',
 'cls_gnames',
 'fivekb_gname',
 'sid',
 'cls_ids',
 'cls_dist',
 '5kb_gid',
 'eqtl_loops',
 'closest_loops',
 'fivekb_loops']

In [37]:
data = data.loc[:, new_order]

In [38]:
master_fn = 'results/main/loop_analysis/super_master.snp_gene_loop.analysis.tsv'
data.to_csv(master_fn, sep='\t', index=False)

In [39]:
master_fn = 'results/main/loop_analysis/super_master.snp_gene_loop.analysis.xlsx'
xdata = data.sort_values(['rs_id', 'dice_cline', 'eqtl_cline', 'study'])
xdata.set_index(['rs_id', 'dice_cline', 'eqtl_cline', 'study']).to_excel(master_fn)